## Pacientes qSOFA 2 e com sepse

In [ ]:
import pandas as pd
df_pacientes = pd.read_csv('hosp/patients.csv')
df_admissoes = pd.read_csv('hosp/admissions.csv')
df_diagnosticos = pd.read_csv('hosp/diagnoses_icd.csv')
df_d_diagnosticos = pd.read_csv("hosp/d_icd_diagnoses.csv")

df_estadia_ed = pd.read_csv('D:\MIMIC\mimic-iv-ed-2.2\ed\edstays.csv')[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime']]
df_diagnosticos_ed = pd.read_csv('D:\MIMIC\mimic-iv-ed-2.2\ed\diagnosis.csv')[['subject_id', 'stay_id', 'icd_code', 'icd_title']]
df_sinais_vitais_ed = pd.read_csv('D:/MIMIC/mimic-iv-ed-2.2/ed/vitalsign.csv')[['subject_id', 'stay_id', 'charttime','temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'rhythm']]

# df_d_items = spark.read.option("header",True).csv("icu/d_items.csv")
# df_chartevents = spark.read.option("header",True).csv("icu/chartevents.csv")
# df_charteventss = df_chartevents.join(df_d_items,df_chartevents.itemid ==  df_d_items.itemid,"inner")

In [ ]:
import pandas as pd
df_diag_sepse = pd.read_excel('pacientes_diag_sepse.xlsx', index_col=0)

In [ ]:
df_diag_sepse

In [ ]:
df_final = df_admissoes.merge(df_pacientes, how='left', on='subject_id')

In [ ]:
df_final

In [ ]:
df_final = df_final.merge(df_diag_sepse[['subject_id','hadm_id']], on= ['subject_id','hadm_id'])

In [ ]:
df_final

In [ ]:
#df_final = df_final[['subject_id', 'hadm_id','gender', 'anchor_age', 'dod', 'admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime', 'icd_code', 'icd_version_x', 'long_title']]
df_final = df_final[['subject_id', 'hadm_id', 'gender', 'anchor_age', 'dod', 'admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime']]

In [ ]:
df_estadia_ed

In [ ]:
df_final =  df_final.merge(df_estadia_ed, how='left', on=['subject_id', 'hadm_id'])

In [ ]:
df_final = df_final.merge(df_sinais_vitais_ed, how='left', on=['subject_id', 'stay_id'])

In [ ]:
columns_transformed = ['subject_id', 'hadm_id', 'Tipo', 'Coluna', 'Timestamp']
df_transformed = pd.DataFrame(columns=columns_transformed)

for _, row in df_final[(df_final['subject_id']==10000032)&(df_final['hadm_id']==29079034)][['subject_id','hadm_id','charttime','resprate','sbp']].iterrows():
    resprate_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'Tipo': 'A frequência respiratória do paciente em respirações por minuto.',
        'Coluna': 'resprate',
        'Timestamp': row['charttime']
    }
    sbp_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'Tipo': 'A pressão arterial sistólica do paciente, medida em milímetros de mercúrio (mmHg).',
        'Coluna': 'sbp',
        'Timestamp': row['charttime']
    }
    df_transformed = df_transformed.append(resprate_row, ignore_index=True)
    df_transformed = df_transformed.append(sbp_row, ignore_index=True)

In [ ]:
# Calcular o a duração do paciente, em dias no hospital
df_final['admittime'] = pd.to_datetime(df_final['admittime'])
df_final['dischtime'] = pd.to_datetime(df_final['dischtime'])
diferenca = df_final['dischtime'] - df_final['admittime']
df_final['LOS hosp'] = diferenca / pd.Timedelta('1 day')

In [ ]:
df_final

In [ ]:
df_final = df_final[(df_final['resprate'].notna()) & (df_final['sbp'].notna())]

In [ ]:
pd.options.mode.chained_assignment = None
df_final['pont_resprate'] = df_final['resprate'].apply(lambda x: 1 if (x if (type(x) == float) else 0) >=22 else 0)
df_final['pont_sbp'] = df_final['sbp'].apply(lambda x: 1 if (x if (type(x) == float) else 200) <=100 else 0)

In [ ]:
df_final_group = df_final.groupby(['subject_id', 'hadm_id'])[['pont_resprate', 'pont_sbp']].max()
df_final_group['pont_total'] = df_final_group['pont_resprate'] + df_final_group['pont_sbp']

In [ ]:
df_final_group

In [ ]:
# Pacientes com sepsis em internações específicas
multi_indice = df_final_group[df_final_group['pont_total']==2].index

In [ ]:
df_pacientes_qSOFA_2_sepse = pd.DataFrame.from_records(multi_indice, columns=['subject_id', 'hadm_id'])
df_pacientes_qSOFA_2_sepse

In [ ]:
len(df_pacientes_qSOFA_2_sepse['subject_id'].unique())

In [ ]:
len(df_pacientes_qSOFA_2_sepse['hadm_id'].unique())

In [ ]:
df_pacientes_qSOFA_2_sepse.to_excel('pacientes_sepse_qSOFA_2.xlsx')

## Pacientes qSOFA 2

In [ ]:
import pandas as pd
df_pacientes = pd.read_csv('hosp/patients.csv')
df_admissoes = pd.read_csv('hosp/admissions.csv')
df_diagnosticos = pd.read_csv('hosp/diagnoses_icd.csv')
df_d_diagnosticos = pd.read_csv("hosp/d_icd_diagnoses.csv")

df_estadia_ed = pd.read_csv('D:\MIMIC\mimic-iv-ed-2.2\ed\edstays.csv')[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime']]
df_diagnosticos_ed = pd.read_csv('D:\MIMIC\mimic-iv-ed-2.2\ed\diagnosis.csv')[['subject_id', 'stay_id', 'icd_code', 'icd_title']]
df_sinais_vitais_ed = pd.read_csv('D:/MIMIC/mimic-iv-ed-2.2/ed/vitalsign.csv')[['subject_id', 'stay_id', 'charttime','temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'rhythm']]

In [ ]:
df_final = df_admissoes.merge(df_pacientes, how='left', on='subject_id')

df_final = df_final[['subject_id', 'hadm_id', 'gender', 'anchor_age', 'dod', 'admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime']]

df_final =  df_final.merge(df_estadia_ed, how='left', on=['subject_id', 'hadm_id'])

df_final = df_final.merge(df_sinais_vitais_ed, how='left', on=['subject_id', 'stay_id'])

In [ ]:
df_final

In [ ]:
columns_transformed = ['subject_id', 'hadm_id', 'Tipo', 'Coluna', 'Timestamp']
df_transformed = pd.DataFrame(columns=columns_transformed)

for _, row in df_final[(df_final['subject_id']==10000032)&(df_final['hadm_id']==29079034)][['subject_id','hadm_id','charttime','resprate','sbp']].iterrows():
    resprate_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'Tipo': 'A frequência respiratória do paciente em respirações por minuto.',
        'Coluna': 'resprate',
        'Timestamp': row['charttime']
    }
    sbp_row = {
        'subject_id': row['subject_id'],
        'hadm_id': row['hadm_id'],
        'Tipo': 'A pressão arterial sistólica do paciente, medida em milímetros de mercúrio (mmHg).',
        'Coluna': 'sbp',
        'Timestamp': row['charttime']
    }
    df_transformed = df_transformed.append(resprate_row, ignore_index=True)
    df_transformed = df_transformed.append(sbp_row, ignore_index=True)

In [ ]:
# Calcular o a duração do paciente, em dias no hospital
df_final['admittime'] = pd.to_datetime(df_final['admittime'])
df_final['dischtime'] = pd.to_datetime(df_final['dischtime'])
diferenca = df_final['dischtime'] - df_final['admittime']
df_final['LOS hosp'] = diferenca / pd.Timedelta('1 day')

In [ ]:
df_final = df_final[(df_final['resprate'].notna()) & (df_final['sbp'].notna())]

In [ ]:
pd.options.mode.chained_assignment = None
df_final['pont_resprate'] = df_final['resprate'].apply(lambda x: 1 if (x if (type(x) == float) else 0) >=22 else 0)
df_final['pont_sbp'] = df_final['sbp'].apply(lambda x: 1 if (x if (type(x) == float) else 200) <=100 else 0)

In [ ]:
df_final_group = df_final.groupby(['subject_id', 'hadm_id'])[['pont_resprate', 'pont_sbp']].max()
df_final_group['pont_total'] = df_final_group['pont_resprate'] + df_final_group['pont_sbp']

In [ ]:
# Pacientes com sepsis em internações específicas
multi_indice = df_final_group[df_final_group['pont_total']==2].index

df_pacientes_qSOFA_2 = pd.DataFrame.from_records(multi_indice, columns=['subject_id', 'hadm_id'])
df_pacientes_qSOFA_2

In [ ]:
df_pacientes_qSOFA_2.to_excel('pacientes_qSOFA_2.xlsx')

## Pacientes diagóstico sepse

In [ ]:
import pandas as pd
df_diagnosticos = pd.read_csv('hosp/diagnoses_icd.csv')
df_d_diagnosticos = pd.read_csv("hosp/d_icd_diagnoses.csv")
df_diagnosticos =  df_diagnosticos.merge(df_d_diagnosticos, how='left', on = 'icd_code')

In [ ]:
df_diagnosticos['long_title2'] = df_diagnosticos['long_title'].str.lower()

In [ ]:
df_seps = df_diagnosticos[df_diagnosticos['long_title2'].str.contains('seps')]

In [ ]:
df_seps.reset_index(drop=True, inplace=True)

In [ ]:
print(f"Número de pacientes: {len(df_seps['subject_id'].unique())}")
print(f"Número de internações: {len(df_seps['hadm_id'].unique())}")

In [ ]:
# Pegar somente o último diagnóstico
df_seps_final = df_seps.groupby(['subject_id', 'hadm_id']).tail(1)

In [ ]:
df_seps_final.reset_index(drop=True, inplace=True)

In [ ]:
df_seps_final.to_excel('pacientes_diag_sepse.xlsx')

## Filtrar pacientes

In [ ]:
import pandas as pd
import os

# pacientes = pd.read_excel('pacientes_qSOFA_2.xlsx', index_col=0)
pacientes = pd.read_excel('pacientes_sepse_qSOFA_2.xlsx', index_col=0)
def filtrar_pacientes():
    _, _, files = next(os.walk("particoes"))
    file_count = len(files)

    for i in range(1,file_count+1):
        df = pd.read_csv(f'particoes\particao_{str(i)}.csv')
        df_resultante = df.merge(pacientes, on=['subject_id', 'hadm_id'])
        df_resultante.to_csv(f'particoes_filtradas/particao_{str(i)}.csv',index=False)
        print(str(i))


In [ ]:
filtrar_pacientes()